In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import MarkerCluster, HeatMapWithTime
import requests
import time
import networkx as nx
from datetime import datetime
import matplotlib.pyplot as plt
from IPython.display import display
import pandas as pd

In [3]:
df = pd.read_csv('/content/drive/MyDrive/LBS/bike_journeys_noOutliers.csv')
df.head()

,Unnamed: 0,origin_index,destination_index,bike_number,origin,destination,departure_time,arrival_time,origin_station_id,origin_lat,origin_lon,destination_staions_id,destination_lat,destination_lon,duration_min
0,0,26103.0,26104,346696,Margaretengürtel U,Heumühlgasse,2025-05-05 14:10:01.958774,2025-05-05 14:30:02.104473,68583064,48.188489,16.343992,68582758,48.195304,16.363308,20.002428
1,1,19295.0,19296,346241,Muthgasse,Friedensbrücke U,2025-05-05 14:10:01.958774,2025-05-05 14:30:02.104473,86734914,48.252055,16.369822,68584504,48.226825,16.365311,20.002428
2,2,50490.0,50491,348365,Reinprechtsdorfer Brücke,Gumpendorfer Gürtel,2025-05-05 14:10:01.958774,2025-05-05 14:30:02.104473,119165095,48.189633,16.351143,117148042,48.189340,16.338321,20.002428
3,3,3575.0,3576,345263,Westbahnhof S U,Siebensternplatz,2025-05-05 14:10:01.958774,2025-05-05 14:30:02.104473,68589226,48.196810,16.338577,119165301,48.202223,16.352486,20.002428
4,5,432.0,433,345062,Westbahnhof S U,Schwedenplatz,2025-05-05 14:10:01.958774,2025-05-05 15:35:02.786508,68589226,48.196810,16.338577,102158706,48.211721,16.376628,85.013796


#Data cleaning

In [4]:
# Sort journeys by departure_time
df = df.sort_values(by='departure_time').reset_index(drop=True)
print(df.head())

   Unnamed: 0  origin_index  destination_index  bike_number  \
0           0       26103.0              26104       346696   
1           1       19295.0              19296       346241   
2           2       50490.0              50491       348365   
3           3        3575.0               3576       345263   
4           5         432.0                433       345062   

                     origin          destination              departure_time  \
0        Margaretengürtel U         Heumühlgasse  2025-05-05 14:10:01.958774   
1                 Muthgasse     Friedensbrücke U  2025-05-05 14:10:01.958774   
2  Reinprechtsdorfer Brücke  Gumpendorfer Gürtel  2025-05-05 14:10:01.958774   
3           Westbahnhof S U     Siebensternplatz  2025-05-05 14:10:01.958774   
4           Westbahnhof S U        Schwedenplatz  2025-05-05 14:10:01.958774   

                 arrival_time  origin_station_id  origin_lat  origin_lon  \
0  2025-05-05 14:30:02.104473           68583064   48.188489   1

In [5]:
journeys = df
journeys.head()

,Unnamed: 0,origin_index,destination_index,bike_number,origin,destination,departure_time,arrival_time,origin_station_id,origin_lat,origin_lon,destination_staions_id,destination_lat,destination_lon,duration_min
0,0,26103.0,26104,346696,Margaretengürtel U,Heumühlgasse,2025-05-05 14:10:01.958774,2025-05-05 14:30:02.104473,68583064,48.188489,16.343992,68582758,48.195304,16.363308,20.002428
1,1,19295.0,19296,346241,Muthgasse,Friedensbrücke U,2025-05-05 14:10:01.958774,2025-05-05 14:30:02.104473,86734914,48.252055,16.369822,68584504,48.226825,16.365311,20.002428
2,2,50490.0,50491,348365,Reinprechtsdorfer Brücke,Gumpendorfer Gürtel,2025-05-05 14:10:01.958774,2025-05-05 14:30:02.104473,119165095,48.189633,16.351143,117148042,48.189340,16.338321,20.002428
3,3,3575.0,3576,345263,Westbahnhof S U,Siebensternplatz,2025-05-05 14:10:01.958774,2025-05-05 14:30:02.104473,68589226,48.196810,16.338577,119165301,48.202223,16.352486,20.002428
4,5,432.0,433,345062,Westbahnhof S U,Schwedenplatz,2025-05-05 14:10:01.958774,2025-05-05 15:35:02.786508,68589226,48.196810,16.338577,102158706,48.211721,16.376628,85.013796


In [6]:
import json
from google.colab import files
origin_stations = journeys[['origin', 'origin_lat', 'origin_lon']].rename(
    columns={'origin': 'station', 'origin_lat': 'lat', 'origin_lon': 'lon'})
dest_stations = journeys[['destination', 'destination_lat', 'destination_lon']].rename(
    columns={'destination': 'station', 'destination_lat': 'lat', 'destination_lon': 'lon'})

all_stations = pd.concat([origin_stations, dest_stations]).drop_duplicates(subset='station')

stations_geojson = {
    "type": "FeatureCollection",
    "features": []
}

for _, row in all_stations.iterrows():
    stations_geojson["features"].append({
        "type": "Feature",
        "properties": {
            "name": row['station']
        },
        "geometry": {
            "type": "Point",
            "coordinates": [row['lon'], row['lat']]
        }
    })

with open("bike_stations.json", "w") as f:
    json.dump(stations_geojson, f, indent=2)

print(json.dumps(stations_geojson, indent=2))
files.download('bike_stations.json')

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {
        "name": "Margareteng\u00fcrtel U"
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          16.343992,
          48.188489
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {
        "name": "Muthgasse"
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          16.369822,
          48.252055
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {
        "name": "Reinprechtsdorfer Br\u00fccke"
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          16.351143,
          48.189633
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {
        "name": "Westbahnhof S U"
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          16.338577,
          48.19681
        ]
      }
    },
    {
  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Cartographic visualisations


##Flow map

In [ ]:
import folium
from folium import Map, PolyLine, CircleMarker
# Create a unified DataFrame of unique station names with their coordinates
origin_stations = df[['origin', 'origin_lat', 'origin_lon']].drop_duplicates()
origin_stations.columns = ['station_name', 'latitude', 'longitude']

destination_stations = df[['destination', 'destination_lat', 'destination_lon']].drop_duplicates()
destination_stations.columns = ['station_name', 'latitude', 'longitude']

station_coords = pd.concat([origin_stations, destination_stations]).drop_duplicates(subset='station_name')

m = folium.Map(location=[48.2082, 16.3738], zoom_start=13)
for _, row in journeys.iterrows():
    if pd.notna(row['origin_lat']) and pd.notna(row['destination_lat']):
        folium.PolyLine(
            locations=[(row['origin_lat'], row['origin_lon']),
                       (row['destination_lat'], row['destination_lon'])],
            color='blue',
            weight=2,
            opacity=0.6,
        ).add_to(m)
for _, row in station_coords.iterrows():
    if pd.notna(row['latitude']):
        folium.CircleMarker(
            location=(row['latitude'], row['longitude']),
            radius=4,
            fill=True,
            fill_color='green',
            fill_opacity=0.7,
            color=None,
        ).add_to(m)
m

In [ ]:
import geopandas as gpd
from shapely.geometry import LineString
journeys['geometry'] = journeys.apply(
    lambda row: LineString([(row['origin_lon'], row['origin_lat']),
                            (row['dest_lon'], row['dest_lat'])])
    if pd.notna(row['origin_lat']) and pd.notna(row['dest_lat']) else None, axis=1)
journeys_gdf = gpd.GeoDataFrame(journeys.dropna(subset=['geometry']), geometry='geometry', crs='EPSG:4326')
journeys_gdf.to_file("bike_journeys.geojson", driver="GeoJSON")
files.download('bike_journeys.geojson')

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
journeys['departure_time'] = pd.to_datetime(journeys['departure_time'])
journeys['arrival_time'] = pd.to_datetime(journeys['arrival_time'])
journeys['duration_min'] = (journeys['arrival_time'] - journeys['departure_time']).dt.total_seconds() / 60
journeys['geometry'] = journeys.apply(
    lambda row: LineString([
        (row['origin_lon'], row['origin_lat']),
        (row['dest_lon'], row['dest_lat'])
    ]) if pd.notna(row['origin_lat']) and pd.notna(row['dest_lat']) else None, axis=1
)
journeys_gdf = gpd.GeoDataFrame(journeys.dropna(subset=['geometry']), geometry='geometry', crs='EPSG:4326')
journeys_gdf.to_file("bike_journeys_duration.geojson", driver="GeoJSON")
files.download('bike_journeys_duration.geojson')

In [ ]:
journeys_gdf.to_csv("bike_journeys_duration.csv")
files.download("bike_journeys_duration.csv")

In [ ]:
df = journeys.copy()
df['origin_lat'] = df['origin_lat'].round(6)
df['origin_lon'] = df['origin_lon'].round(6)
df['dest_lat'] = df['dest_lat'].round(6)
df['dest_lon'] = df['dest_lon'].round(6)

grouped = df.groupby(['origin_lat', 'origin_lon', 'dest_lat', 'dest_lon']).size().reset_index(name='trip_count')
grouped['geometry'] = grouped.apply(
    lambda row: LineString([
        (row['origin_lon'], row['origin_lat']),
        (row['dest_lon'], row['dest_lat'])
    ]), axis=1
)

flow_gdf = gpd.GeoDataFrame(grouped, geometry='geometry', crs='EPSG:4326')
flow_gdf.to_file("aggregated_bike_flows.geojson", driver="GeoJSON")
files.download('aggregated_bike_flows.geojson')

In [ ]:
flow_gdf.head()
flow_gdf.to_csv("aggregated_bike_flows.csv")
files.download("aggregated_bike_flows.csv")

In [8]:
df = journeys.copy()

df['departure_time'] = pd.to_datetime(df['departure_time'])
df['arrival_time'] = pd.to_datetime(df['arrival_time'])

df['duration_min'] = (df['arrival_time'] - df['departure_time']).dt.total_seconds() / 60

df['origin_lat'] = df['origin_lat'].round(6)
df['origin_lon'] = df['origin_lon'].round(6)
df['dest_lat'] = df['dest_lat'].round(6)
df['dest_lon'] = df['dest_lon'].round(6)

agg = df.groupby(['origin_lat', 'origin_lon', 'dest_lat', 'dest_lon']).agg(
    trip_count=('duration_min', 'count'),
    avg_duration_min=('duration_min', 'mean')
).reset_index()

agg['geometry'] = agg.apply(
    lambda row: LineString([
        (row['origin_lon'], row['origin_lat']),
        (row['dest_lon'], row['dest_lat'])
    ]), axis=1
)

agg_gdf = gpd.GeoDataFrame(agg, geometry='geometry', crs='EPSG:4326')

agg_gdf.to_file("agg_dur_traj.geojson", driver="GeoJSON")

KeyError: 'dest_lat'

##Comparing time of day

In [ ]:
morning = journeys[
    (journeys['departure_time'].dt.hour >= 7) &
    (journeys['departure_time'].dt.hour < 11)
]
midday = journeys[
    (journeys['departure_time'].dt.hour >= 11) &
    (journeys['departure_time'].dt.hour < 16)
]
evening = journeys[
    (journeys['departure_time'].dt.hour >= 16) &
    (journeys['departure_time'].dt.hour < 20)
]
night = journeys[
    (journeys['departure_time'].dt.hour >= 20) |
    (journeys['departure_time'].dt.hour < 7)
]

In [ ]:
def aggregate_od(df, label):
    df = df.copy()
    df['duration_min'] = (df['arrival_time'] - df['departure_time']).dt.total_seconds() / 60
    df['origin_lat'] = df['origin_lat'].round(6)
    df['origin_lon'] = df['origin_lon'].round(6)
    df['dest_lat'] = df['dest_lat'].round(6)
    df['dest_lon'] = df['dest_lon'].round(6)
    agg = df.groupby(['origin', 'destination', 'origin_lat', 'origin_lon', 'dest_lat', 'dest_lon']).agg(
        trip_count=('duration_min', 'count'),
        avg_duration_min=('duration_min', 'mean')
    ).reset_index()
    agg.to_csv(f"{label}_od_summary.csv", index=False)
    return agg

journeys['departure_time'] = pd.to_datetime(journeys['departure_time'])
journeys['arrival_time'] = pd.to_datetime(journeys['arrival_time'])

morning = journeys[(journeys['departure_time'].dt.hour >= 7) & (journeys['departure_time'].dt.hour < 11)]
midday  = journeys[(journeys['departure_time'].dt.hour >= 11) & (journeys['departure_time'].dt.hour < 16)]
evening = journeys[(journeys['departure_time'].dt.hour >= 16) & (journeys['departure_time'].dt.hour < 20)]
night   = journeys[(journeys['departure_time'].dt.hour >= 20) | (journeys['departure_time'].dt.hour < 7)]

morning_od = aggregate_od(morning, "morning")
midday_od  = aggregate_od(midday, "midday")
evening_od = aggregate_od(evening, "evening")
night_od   = aggregate_od(night, "night")
#files.download("morning_od_summary.csv")
#files.download("midday_od_summary.csv")
#files.download("evening_od_summary.csv")
#files.download("night_od_summary.csv")
morning_od.head()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,origin,destination,origin_lat,origin_lon,dest_lat,dest_lon,trip_count,avg_duration_min
0,Absberggasse,Erdberg U,48.174848,16.390250,48.189551,16.418756,1,24.983532
1,Absberggasse,Hauptbahnhof Ost,48.174848,16.390250,48.184498,16.380578,1,10.007998
2,Absberggasse,Kundratstraße,48.174848,16.390250,48.175187,16.350095,1,19.977088
3,Absberggasse,Pilgramgasse U,48.174848,16.390250,48.191400,16.353798,2,117.507145
4,Adelheid-Popp-Gasse,Aspernstraße U,48.229232,16.458945,48.223565,16.476992,1,14.982713


##Visualising departure vs arrival stations

In [ ]:
import pandas as pd

def classify_station_activity(df):
    df = df.dropna(subset=[
        'origin', 'origin_lat', 'origin_lon',
        'destination', 'dest_lat', 'dest_lon'
    ])

    dep = df.groupby(['origin', 'origin_lat', 'origin_lon']).size().reset_index(name='dep_count')
    arr = df.groupby(['destination', 'dest_lat', 'dest_lon']).size().reset_index(name='arr_count')
    dep = dep.rename(columns={'origin': 'station', 'origin_lat': 'lat', 'origin_lon': 'lon'})
    arr = arr.rename(columns={'destination': 'station', 'dest_lat': 'lat', 'dest_lon': 'lon'})
    stations = pd.merge(dep, arr, on=['station', 'lat', 'lon'], how='outer').fillna(0)
    stations['activity_type'] = stations.apply(
        lambda row: (
            'more_arrivals' if row['arr_count'] > row['dep_count']
            else 'more_departures' if row['dep_count'] > row['arr_count']
            else 'equal'
        ),
        axis=1
    )

    return stations

station_activity = classify_station_activity(journeys)
station_activity.to_csv("station_dep_vs_arr.csv", index=False)
files.download("station_dep_vs_arr.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Adding a division by section of day
import pandas as pd

def station_temporal_classification(df):
    df = df.dropna(subset=[
        'origin', 'origin_lat', 'origin_lon',
        'destination', 'dest_lat', 'dest_lon'
    ])

    df['departure_time'] = pd.to_datetime(df['departure_time'])
    df['arrival_time'] = pd.to_datetime(df['arrival_time'])
    def get_period(hour):
        if 7 <= hour < 11:
            return 'morning'
        elif 11 <= hour < 16:
            return 'midday'
        elif 16 <= hour < 20:
            return 'evening'
        else:
            return 'night'

    df['dep_period'] = df['departure_time'].dt.hour.apply(get_period)
    df['arr_period'] = df['arrival_time'].dt.hour.apply(get_period)

    dep = df.groupby(['origin', 'origin_lat', 'origin_lon', 'dep_period']).size().unstack(fill_value=0).reset_index()
    dep = dep.rename(columns={
        'origin': 'station', 'origin_lat': 'lat', 'origin_lon': 'lon',
        'morning': 'dep_morning', 'midday': 'dep_midday',
        'evening': 'dep_evening', 'night': 'dep_night'
    })

    arr = df.groupby(['destination', 'dest_lat', 'dest_lon', 'arr_period']).size().unstack(fill_value=0).reset_index()
    arr = arr.rename(columns={
        'destination': 'station', 'dest_lat': 'lat', 'dest_lon': 'lon',
        'morning': 'arr_morning', 'midday': 'arr_midday',
        'evening': 'arr_evening', 'night': 'arr_night'
    })

    stations = pd.merge(dep, arr, on=['station', 'lat', 'lon'], how='outer').fillna(0)

    def classify(dep, arr):
        if arr > dep:
            return 'more_arrivals'
        elif dep > arr:
            return 'more_departures'
        else:
            return 'equal'

    for period in ['morning', 'midday', 'evening', 'night']:
        stations[f'status_{period}'] = stations.apply(
            lambda row: classify(row.get(f'dep_{period}', 0), row.get(f'arr_{period}', 0)),
            axis=1
        )

    columns = (
        ['station', 'lat', 'lon'] +
        [f'dep_{p}' for p in ['morning', 'midday', 'evening', 'night']] +
        [f'arr_{p}' for p in ['morning', 'midday', 'evening', 'night']] +
        [f'status_{p}' for p in ['morning', 'midday', 'evening', 'night']]
    )
    return stations[columns]

classified_stations = station_temporal_classification(journeys_gdf)
classified_stations.to_csv("station_arr_dep_time.csv", index=False)
files.download("station_arr_dep_time.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Network Analysis

In [ ]:
import networkx as nx
import pandas as pd

def build_bike_network(df):
    flow_df = df.groupby(['origin', 'destination']).size().reset_index(name='weight')
    G = nx.DiGraph()
    for _, row in flow_df.iterrows():
        G.add_edge(row['origin'], row['destination'], weight=row['weight'])
    return G, flow_df

def compute_centralities(G):
    degree_centrality = nx.degree_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G, weight='weight', normalized=True)
    return degree_centrality, betweenness_centrality
G, flows = build_bike_network(journeys_gdf)
deg_cent, bet_cent = compute_centralities(G)

centralities_df = pd.DataFrame({
    'station': list(deg_cent.keys()),
    'degree_centrality': list(deg_cent.values()),
    'betweenness_centrality': list(bet_cent.values())
})

coords = journeys_gdf[['origin', 'origin_lat', 'origin_lon']].rename(
    columns={'origin': 'station', 'origin_lat': 'lat', 'origin_lon': 'lon'}
).drop_duplicates()

centralities_df = pd.merge(centralities_df, coords, on='station', how='left')

origin_counts = journeys_gdf.groupby('origin').size().reset_index(name='trips_started')

destination_counts = journeys_gdf.groupby('destination').size().reset_index(name='trips_ended')

centralities_df = pd.merge(centralities_df, origin_counts.rename(columns={'origin': 'station'}), on='station', how='left')
centralities_df = pd.merge(centralities_df, destination_counts.rename(columns={'destination': 'station'}), on='station', how='left')

centralities_df['trips_started'] = centralities_df['trips_started'].fillna(0).astype(int)
centralities_df['trips_ended'] = centralities_df['trips_ended'].fillna(0).astype(int)

centralities_df = centralities_df.drop('betweenness_centrality', axis=1)
centralities_df['trips'] = centralities_df['trips_started'] + centralities_df['trips_ended']
centralities_df.to_csv('network_analysis.csv', index=False)
files.download('network_analysis.csv')
centralities_df.head()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,station,degree_centrality,lat,lon,trips_started,trips_ended,trips
0,AIT Austrian Institute of Technology / Giefing...,0.056277,48.269137,16.426676,16,20,36
1,Floridsdorf S U,0.337662,48.255701,16.400104,140,144,284
2,Friedrich-Engels-Platz,0.281385,48.244433,16.381087,79,104,183
3,Handelskai S U,0.519481,48.242036,16.384798,193,201,394
4,Josef-Baumann-Gasse,0.121212,48.253030,16.431119,35,32,67


##Space-time cube (Kepler.gl)

In [ ]:
trips_df = journeys[['bike_number', 'departure_time', 'origin_lat', 'origin_lon',
                     'arrival_time', 'dest_lat', 'dest_lon']].copy()
trips_df = trips_df.rename(columns={
    'bike_number': 'bike_id',
    'departure_time': 'departure_time',
    'arrival_time': 'arrival_time',
    'origin_lat': 'origin_lat',
    'origin_lon': 'origin_lon',
    'dest_lat': 'destination_lat',
    'dest_lon': 'destination_lon'
})

trips_df.to_csv('bike_trips.csv', index=False)
files.download('bike_trips.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Create CSV of trip starts
starts_df = journeys[['bike_number', 'departure_time', 'origin_lat', 'origin_lon']].copy()
starts_df = starts_df.rename(columns={
    'bike_number': 'bike_id',
    'departure_time': 'timestamp',
    'origin_lat': 'lat',
    'origin_lon': 'lon'
})

starts_df.to_csv('bike_starts.csv', index=False)
files.download('bike_starts.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>